In [1]:
import re

class LexicalAnalyzer:
    def __init__(self):
        self.token_pattern = re.compile(r'\d+\.?\d*|[+\-*/()]')

    def tokenize(self, text):
        tokens = self.token_pattern.findall(text)
        # Remove whitespace tokens
        tokens = [token for token in tokens if token.strip()]
        return tokens

# Test the Lexical Analyzer
lexer = LexicalAnalyzer()
tokens = lexer.tokenize("(5 + 3) * (10 / 2)")
print("Tokens:", tokens)

Tokens: ['(', '5', '+', '3', ')', '*', '(', '10', '/', '2', ')']


In [2]:
class ArithmeticParser:
    def __init__(self):
        self.tokens = []
        self.current_token_index = 0

    def parse_input(self, text):
        self.tokens = LexicalAnalyzer().tokenize(text)
        self.current_token_index = 0
        result = self.expression()
        if self.current_token_index != len(self.tokens):
            raise ValueError("Invalid expression.")
        return result

    def expression(self):
        return self.addition()

    def addition(self):
        result = self.multiplication()
        while self.match('+') or self.match('-'):
            operator = self.tokens[self.current_token_index - 1]
            right = self.multiplication()
            if operator == '+':
                result += right
            else:
                result -= right
        return result

    def multiplication(self):
        result = self.primary()
        while self.match('*') or self.match('/'):
            operator = self.tokens[self.current_token_index - 1]
            right = self.primary()
            if operator == '*':
                result *= right
            else:
                if right == 0:
                    raise ValueError("Division by zero.")
                # Ensure result is treated as a float
                result = float(result) / right
        return result

    def primary(self):
        token = self.tokens[self.current_token_index]
        self.current_token_index += 1
        if token == '(':
            result = self.expression()
            if self.tokens[self.current_token_index] == ')':
                self.current_token_index += 1
                return result
            else:
                raise ValueError("Expected closing parenthesis.")
        elif re.match(r'\d+\.?\d*', token):
            return float(token)
        else:
            raise ValueError("Invalid expression.")

    def match(self, token_type):
        if self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index] == token_type:
            self.current_token_index += 1
            return True
        return False

# Test the Parser
parser = ArithmeticParser()
result = parser.parse_input("(5 + 3) * (10 / 2)")
print("Result:", result)

Result: 40.0


In [4]:
import re
import ipywidgets as widgets
from IPython.display import display, HTML

# Function to check syntax errors
def check_syntax(code):
    syntax_errors = []
    # Regular expression to find common syntax errors
    syntax_patterns = [
        (r'\bfor\b.*\bin\b', "Use 'in' instead of 'in'"),
        (r'\bif\b.*?:', "Missing colon after 'if' statement"),
        (r':\n\s*\n', "Remove extra empty lines after colon"),
        # Add more syntax patterns as needed
    ]
    for pattern, error_msg in syntax_patterns:
        if re.search(pattern, code):
            syntax_errors.append(error_msg)
    return syntax_errors

# Function to execute the code
def execute_code(code):
    try:
        result = eval(code)
        return result
    except Exception as e:
        return str(e)

# Event handler for check syntax button
def on_check_syntax_button_clicked(b):
    syntax_errors = check_syntax(text_box.value)
    if syntax_errors:
        error_logger.value = "\n".join(syntax_errors)
    else:
        error_logger.value = "No syntax errors found."

# Event handler for execute button
def on_execute_button_clicked(b):
    result = execute_code(text_box.value)
    if isinstance(result, str):
        error_logger.value = result
    else:
        error_logger.value = "Result: " + str(result)

# Text box for writing code
text_box = widgets.Textarea(
    placeholder='Write your code here...',
    layout=widgets.Layout(width='50%', height='200px')
)

# Check syntax button
check_syntax_button = widgets.Button(description='Check Syntax')
check_syntax_button.on_click(on_check_syntax_button_clicked)

# Execute button
execute_button = widgets.Button(description='Execute')
execute_button.on_click(on_execute_button_clicked)

# Error logger
error_logger = widgets.Textarea(
    placeholder='Error log...',
    layout=widgets.Layout(width='50%', height='100px')
)

# Display widgets
display(text_box)
display(check_syntax_button)
display(execute_button)
display(error_logger)


Textarea(value='', layout=Layout(height='200px', width='50%'), placeholder='Write your code here...')

Button(description='Check Syntax', style=ButtonStyle())

Button(description='Execute', style=ButtonStyle())

Textarea(value='', layout=Layout(height='100px', width='50%'), placeholder='Error log...')